In [ ]:
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time


In [ ]:
HEADER = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
URL = "https://weknife.com/plus/list-2.html"
#PROXY = {"https": "https//59.110.7.190:1080"}


In [ ]:

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(URL)
browser.implicitly_wait(60) 
for i in range(1,100):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    html_source = browser.page_source
    data = html_source.encode('utf-8')
#python_button = browser.find_element_by_xpath('//*[@id="maincontent"]/div[2]/div/div[2]/ol/li[31]/div/p[1]')
#python_button.click()

In [ ]:
Bootifulobj = soup(browser.page_source, "html.parser")
#Bootifulobj


In [ ]:
knife_items = Bootifulobj.find_all('div',class_= "item")
len(knife_items)


In [ ]:
Price= []
for P in Bootifulobj.findAll('div',{'class': "msrp" }):
    #print(p.text)
    Price.append(P.text)
len(Price)

In [ ]:
Title = []
for T in Bootifulobj.findAll('div',{'class': "producttitle3"}):
    #print(p.text)
    Title.append(T.text)
len(Title)

In [ ]:
title_price = pd.DataFrame(zip(Title, Price))
title_price

In [ ]:
product_links= []
baseurl = 'https://weknife.com'
for items in knife_items:
    for links in items.find_all('a', href = True, title =True):
        product_links.append(baseurl + links['href'])

In [ ]:
# dataframe that will hold the final resulting data\,
final = pd.DataFrame()
for links in product_links:
    HTML2 = requests.get(links, HEADER)
    Booti2 = soup(HTML2.content,"lxml")
    try:
        buffer = pd.DataFrame(
            [[
                #name\n",
                Booti2.find('div', class_='newttitle').text.strip(),
                str(Booti2.find('div', class_ = 'detailContent').find(text = re.compile(r'(?:Blade Style: )'))[13:].replace('Blade Style: ', '')),
            ]], 
            
            columns= ['Name', 'Blade Style']#, 'Overall Length']#, 'Closed Length','Blade Thickness', 'Handle Thickness', 'Weight', 'Blade Material', 'Blade Hardness', 'Blade Grind', 'Blade Finish', 'Handle Material', 'Handle Onlay', 'Handle Color/Finish', 'Back Spacer Material', 'Liner Material', 'Screws Material', 'Pivot Assembly', 'Locking Mechanism', 'Sheath', 'Designer']
            )
        final = final.append(buffer)
    except:
        continue
    print(final)
    print(len(final))

In [ ]:
final.to_excel("wiknife.xlsx")

In [ ]:
Knives_List = []
testlinks = "https://weknife.com/plus/view-234-1.html"  
for links in product_links:
    HTML2 = requests.get(links, HEADER)
    Booti2 = soup(HTML2.content,"lxml")
#print(Booti2)
    try:
        Title = Booti2.find('meta', {'name':'keywords'}).get('content')
        Desc = Booti2.find('meta', {'name':'description'}).get('content')
    except:
        continue
    We_Knives_Desc = {'Title':Title,
                        'Desc':  Desc
                        }
    Titles_Desc.append(We_Knives_Desc)

In [ ]:

Titles_Desc = pd.DataFrame(Titles_Desc)
Price = pd.DataFrame(Price)
Knives = [Titles_Desc, Price]
Knives_List_Final = pd.concat(Knives, axis = 1)
len(Knives_List_Final)

In [ ]:
Knives_List_Final.to_csv('we_knife.csv',index=False, mode='a')


In [ ]:
type(Knives_List_Final.iloc[0,1])
